In [1]:
reset -f -s

In [2]:
import os
ROOTDIR = os.path.abspath(os.path.dirname('__file__'))
DATADIR = os.path.join(ROOTDIR, 'data')

In [3]:
print(ROOTDIR)
print(DATADIR)

/Users/anqishen/Fake_News_AS
/Users/anqishen/Fake_News_AS/data


In [4]:
import pandas as pd

df = pd.read_csv(os.path.join(DATADIR, 'fake_or_real_news.csv'))
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
!conda install -c conda-forge -y gensim
import nltk
from nltk.corpus import gutenberg
from gensim import models

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/anqishen/anaconda3:
#
gensim                    3.1.0            py36h33de7db_0  


In [6]:
import nltk
# Transform each review string as a list of token strings. May take a few seconds
texts = df.text.values
tokenized = [nltk.word_tokenize(reviews) for reviews in texts]

In [7]:
import nltk
from string import punctuation

def clean_text(tokenized_list, sw):
    new_list = []
    for doc in tokenized_list:
        new_list.append([token.lower() for token in doc if token.lower() not in sw])
    return new_list

In [8]:
from string import punctuation
texts = df.text
mapping_table = {ord(char): u' ' for char in punctuation}
tokenized = [nltk.word_tokenize(reviews.translate(mapping_table)) for reviews in texts]

In [9]:
from nltk.corpus import stopwords
sw = stopwords.words('english')
cleaned = clean_text(tokenized, sw)

In [10]:
from nltk.corpus import gutenberg
from gensim import models

# Training word2vec model on Gutenberg corpus. This may take a few minutes.
word2vec = models.Word2Vec(cleaned,
                        size = 300,
                        window = 5,
                        min_count = 5,
                        sg = 0,
                        alpha = 0.025,
                        iter=10,
                        batch_words = 10000)

In [11]:
class TagDocIterator:
    def __init__(self, doc_list, idx_list):
        self.doc_list = doc_list
        self.idx_list = idx_list
        
    def __iter__(self):
        for doc, idx, in zip(self.doc_list, self.idx_list):
            tag = [idx]
            yield TaggedDocument(words=doc, tags=tag)

In [12]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import reuters


# Create and train the doc2vec model. May take a few seconds
doc2vec = Doc2Vec(size=300, 
                  window=5,
                  min_count=5,
                  dm = 1,
                  iter=5)

# Build the word2vec model from the corpus
doc2vec.build_vocab(TagDocIterator(cleaned, df.index))

/Users/anqishen/.local/lib/python3.6/site-packages/gensim/models/doc2vec.py:355: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Users/anqishen/.local/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [13]:
doc2vec.train(TagDocIterator(cleaned, df.index), epochs = 5, total_examples = doc2vec.corpus_count)

In [14]:
doc2vec.docvecs.most_similar(positive=70)

[(5595, 0.8306317329406738),
 (3580, 0.80402010679245),
 (3166, 0.7926143407821655),
 (4056, 0.7886194586753845),
 (5491, 0.7818440794944763),
 (4366, 0.7777795791625977),
 (2258, 0.7746214866638184),
 (2181, 0.7719109058380127),
 (2393, 0.7665035128593445),
 (1331, 0.7582512497901917)]

In [15]:
#This may take a few minutes to run
w2v_loc = '/Users/anqishen/Fake_News_AS/data/GoogleNews-vectors-negative300.bin.gz'
word2vec.intersect_word2vec_format(w2v_loc, binary=True)

In [16]:
import numpy as np
X = np.empty([df.shape[0],300])
for i in range(df.shape[0]):
    X[i] = doc2vec.docvecs[i]

In [17]:
import numpy as np
def binarize(word):
    if word  == 'REAL':
        return 0
    else:
        return 1
y = np.asarray(df.label.apply(binarize))
y = y.reshape([df.shape[0],])

In [18]:
#forming training sets and test sets
from sklearn.model_selection import train_test_split
seed = 42
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
print(y_train.shape)
print(X_train.shape)

(4244,)
(4244, 300)


In [19]:
#use logistic classifier
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
print(type(X_train))
print(type(y_train))
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)

model.score(X_test, y_test)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


0.8574844571975132

In [ ]:
#reference: Week 2 lecture notebook; Lab 2 notebook